In [1]:
import numpy as np
import matplotlib.pyplot as plt
import csv
import numpy.linalg as LA
import random
import pandas as pd
from numpy import inf
import torch
from torch import nn
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
import torch.optim as optim
import json
import itertools
import math
import pyprind
from random import sample

In [2]:
f = open('../data_irl_model/Feb23_data_exploratory_policy.json')
#f = open('../data_irl_model/Feb22_dataSVO.json')
data_irl = json.load(f)

In [3]:
data_keys=list(data_irl.keys())
data_keys.sort()

keys_for_training=[]
keys_for_testing=[]

index=0
for vehicle in data_keys:
    if index%10==0:
        keys_for_testing.append(vehicle)
        index+=1
    else:
        keys_for_training.append(vehicle)
        index+=1

In [6]:
key=keys_for_training[97]
key=str(key)                
############# Create the data for the training locally- we are restricting the number of timesteps to num_timesteps######
negative_vals=[]
for i in [0,1]:
    negative_vals.append(list(filter(lambda x: x < 1, data_irl[key][i])))
num_timesteps=min(50,len(negative_vals[0]),len(negative_vals[1]))

print("0: ", negative_vals[0])
print("1: ", negative_vals[1])

print("num_timesteps: ", num_timesteps)

key=keys_for_training[159]

key=str(key)  

negative_vals=[]

for i in [0,1]:
    negative_vals.append(list(filter(lambda x: x < 1, data_irl[key][i])))

num_timesteps=min(50,len(negative_vals[0]),len(negative_vals[1]))

index_before_zero=data_irl[key][1].index(negative_vals[1][len(negative_vals[1])-1])

print(index_before_zero)

0:  [-50.0, -48.00000959491307, -46.000038513319524, -44.000087269879714, -42.000156438656006, -40.00024642268436, -38.00035769377065, -36.00049081444543, -34.00064644271893, -32.00082533931872, -30.00102840504124, -28.00125668872575, -26.00151120767363, -24.001792382791493, -22.002099833316123, -20.002432376013743, -18.002787437319792, -16.003160131001316, -14.003541860606148, -12.003918337158034, -10.004267320991964, -8.004557419881444, -6.004750413867271, -3.9448847480055775, -1.765019082143887, 0.5348465837178011]
1:  [-64.01737173704754, -62.83531527736229, -61.533258817677044, -60.1112023579918, -58.56914589830655, -56.907089438621306, -55.12503297893606, -53.222976519250814, -51.20092005956557, -49.05886359988032, -46.79680714019507, -44.435783573886155, -42.03579318943008, -39.63580342381655, -37.23581436583025, -34.83582611992979, -32.435838815383825, -30.035852614743888, -27.635867726241045, -25.23588442248834, -22.83590306972686, -20.435924175542993, -18.035948470786764, -15

In [4]:
#########################################################################
################################ AIS_gen ################################

################################ Model 1 ################################ 

class GenerateAis1(nn.Module):

    #def __init__(self,n_input,n_state,n_psi2_in=64,n_psi2_out=128):
    def __init__(self,n_input,n_state,n_psi2_in=8,n_psi2_out=16):
        super(GenerateAis1,self).__init__()
        self.PSI_layer1=nn.Linear(n_input,n_psi2_in)     # Use RELU after
        self.PSI_layer2=nn.Linear(n_psi2_in,n_psi2_out)      # Use RELU after
        self.PSI_layer3=nn.GRUCell(n_psi2_out,n_state)       # This is the Gated layer

    def forward(self,x,h):
        #x=torch.transpose(x,0,1)
        #h=torch.transpose(h,0,1)
        x=torch.relu(self.PSI_layer1(x))
        x=torch.relu(self.PSI_layer2(x))
        h=self.PSI_layer3(x,h)
        return h     


In [5]:
#########################################################################
################################ AIS_pred ###############################

################################ Model 1 ################################ 

class PredictAis1(nn.Module):
    
    #def __init__(self,n_output,n_state,n_phi2_in=16,n_phi2_out=8):
    def __init__(self,n_output,n_decoder_in,n_phi2_in=6,n_phi2_out=8):
        super(PredictAis1,self).__init__()
        self.PHI_layer1=nn.Linear(n_decoder_in,n_phi2_in)  # Use RELU after
        self.PHI_layer2=nn.Linear(n_phi2_in,n_phi2_out)     # Use RELU after
        self.PHI_layer3=nn.Linear(n_phi2_out,n_output)         # mean vector of a unit-variance multivariate Gaussian distribution, samples from which are used to predict the next observation

    # x here is the hidden state and the current action that is chosen 
    # to predict the next observations for the horizon
    def forward(self,x):
        x=torch.relu(self.PHI_layer1(x))
        x=torch.relu(self.PHI_layer2(x))
        output=self.PHI_layer3(x)
        return output


In [7]:
class NN_structure(object):
    def __init__(self,n_epochs,min_timesteps,n_rollout,n_skips_per_rollout,n_test,n_input,n_output,n_state_enc,learning_rate,ais_gen_model,ais_pred_model,device):
        self.n_epochs=n_epochs
        self.min_timesteps=min_timesteps
        self.rollout=n_rollout
        self.n_skips=n_skips_per_rollout
        self.n_test=n_test
        self.n_input=n_input
        self.n_output=n_output
        self.n_state_enc=n_state_enc    # Hidden state size in RNN
        self.learning_rate=learning_rate                
        self.device=device
        self.ais_gen_model=ais_gen_model
        self.ais_pred_model=ais_pred_model

        self.n_psi2_in=8
        self.n_psi2_out=16
        self.n_phi2_in=2
        self.n_phi2_out=4
        self.n_psi1_out=8
        
        if ais_gen_model==1:
            self.gen_model=GenerateAis1(self.n_input,self.n_state_enc).to(self.device)      
        
        #if ais_gen_model==2: 
            #self.gen_model=GenerateAis2(self.n_input,self.n_state_enc,self.n_psi1_out).to(self.device)        
            #self.gen_model=GenerateAis2_LSTM(self.n_input,self.n_state_enc,self.n_psi1_out).to(self.device)
            #self.gen_model=GenerateAis_LSTMShallow(self.n_input,self.n_state_enc).to(self.device)  

        if ais_pred_model==1:
            self.pred_model=PredictAis1(self.n_output,self.n_state_enc+1).to(self.device) # self.n_state_enc+1 implies ais_t and the (action of cav)_t
        
        #if ais_pred_model==2:
            #self.pred_model=PredictAis2(self.n_output,self.n_state_enc+1,self.n_state_dec).to(self.device)
            #self.pred_model=PredictAis2_LSTM(self.n_output,self.n_state_enc+1,self.n_state_dec).to(self.device)
            #self.pred_model=PredictAis_LSTMShallow(self.n_state_enc+1,self.n_state_dec).to(self.device)

    def save_model_weights(self,text="(:*_*:)"):
        #name_gen_path="trained_models/IRL_based_models/IRL_SVOpolicy_ais_gen"+str(self.ais_gen_model)+"_"+str(self.n_psi2_in)+str(self.n_psi2_out)+"_pred"+str(self.ais_pred_model)+"_"+str(self.n_phi2_in)+str(self.n_phi2_out)+"_epochs"+str(self.n_epochs)+"_learning_rate"+str(self.learning_rate)+"_hidden_states"+str(self.n_state_enc)+text+".pth"
        #torch.save(self.gen_model.state_dict(),name_gen_path)
        
        name_gen_path="_"
        name_pred_path="_"
        if self.ais_pred_model==1:
            name_pred_path="../trained_models/IRL_based_models/IRL_explorepolicy_ais_pred"+str(self.ais_pred_model)+"_"+str(self.n_phi2_in)+str(self.n_phi2_out)+"_gen"+str(self.ais_gen_model)+"_"+str(self.n_psi2_in)+str(self.n_psi2_out)+"_epochs"+str(self.n_epochs)+"_learning_rate"+str(self.learning_rate)+"_hidden_states"+str(self.n_state_enc)+text+".pth"
            name_gen_path="../trained_models/IRL_based_models/IRL_explorepolicy_ais_gen"+str(self.ais_gen_model)+"_"+str(self.n_psi2_in)+str(self.n_psi2_out)+"_pred"+str(self.ais_pred_model)+"_"+str(self.n_phi2_in)+str(self.n_phi2_out)+"_epochs"+str(self.n_epochs)+"_learning_rate"+str(self.learning_rate)+"_hidden_states"+str(self.n_state_enc)+text+".pth"

        if self.ais_pred_model==2:
            name_pred_path="../trained_models/IRL_based_models/IRL_explorepolicy_multi_rnn_ais_pred"+str(self.ais_pred_model)+"_"+str(self.n_phi2_in)+str(self.n_phi2_out)+"_gen"+str(self.ais_gen_model)+"_"+str(self.n_psi2_in)+str(self.n_psi2_out)+"_epochs"+str(self.n_epochs)+"_learning_rate"+str(self.learning_rate)+"_hidden_states"+str(self.n_state_enc)+"_"+str(self.n_state_dec)+text+".pth"
            name_gen_path="../trained_models/IRL_based_models/IRL_explorepolicy_multi_rnn_ais_gen"+str(self.ais_gen_model)+"_"+str(self.n_psi2_in)+str(self.n_psi2_out)+"_pred"+str(self.ais_pred_model)+"_"+str(self.n_phi2_in)+str(self.n_phi2_out)+"_epochs"+str(self.n_epochs)+"_learning_rate"+str(self.learning_rate)+"_hidden_states"+str(self.n_state_enc)+"_"+str(self.n_state_dec)+text+".pth"

        torch.save(self.gen_model.state_dict(),name_gen_path)
        torch.save(self.pred_model.state_dict(),name_pred_path)

        return "saved"
    
    def load_model_weights(self,text="(:*_*:)"):
        #name_gen_path="trained_models/IRL_based_models/IRL_SVOpolicy_ais_gen"+str(self.ais_gen_model)+"_"+str(self.n_psi2_in)+str(self.n_psi2_out)+"_pred"+str(self.ais_pred_model)+"_"+str(self.n_phi2_in)+str(self.n_phi2_out)+"_epochs"+str(self.n_epochs)+"_learning_rate"+str(self.learning_rate)+"_hidden_states"+str(self.n_state_enc)+text+".pth"
        #torch.save(self.gen_model.state_dict(),name_gen_path)
        #self.gen_model.load_state_dict(torch.load(name_gen_path))
        #self.gen_model.eval()

        name_gen_path="_"
        name_pred_path="_"
        
        if self.ais_pred_model==1:
            name_pred_path="../trained_models/IRL_based_models/IRL_explorepolicy_ais_pred"+str(self.ais_pred_model)+"_"+str(self.n_phi2_in)+str(self.n_phi2_out)+"_gen"+str(self.ais_gen_model)+"_"+str(self.n_psi2_in)+str(self.n_psi2_out)+"_epochs"+str(self.n_epochs)+"_learning_rate"+str(self.learning_rate)+"_hidden_states"+str(self.n_state_enc)+text+".pth"
            name_gen_path="../trained_models/IRL_based_models/IRL_explorepolicy_ais_gen"+str(self.ais_gen_model)+"_"+str(self.n_psi2_in)+str(self.n_psi2_out)+"_pred"+str(self.ais_pred_model)+"_"+str(self.n_phi2_in)+str(self.n_phi2_out)+"_epochs"+str(self.n_epochs)+"_learning_rate"+str(self.learning_rate)+"_hidden_states"+str(self.n_state_enc)+text+".pth"

        if self.ais_pred_model==2:
            name_pred_path="../trained_models/IRL_based_models/IRL_explorepolicy_multi_rnn_ais_pred"+str(self.ais_pred_model)+"_"+str(self.n_phi2_in)+str(self.n_phi2_out)+"_gen"+str(self.ais_gen_model)+"_"+str(self.n_psi2_in)+str(self.n_psi2_out)+"_epochs"+str(self.n_epochs)+"_learning_rate"+str(self.learning_rate)+"_hidden_states"+str(self.n_state_enc)+"_"+str(self.n_state_dec)+text+".pth"
            name_gen_path="../trained_models/IRL_based_models/IRL_explorepolicy_multi_rnn_ais_gen"+str(self.ais_gen_model)+"_"+str(self.n_psi2_in)+str(self.n_psi2_out)+"_pred"+str(self.ais_pred_model)+"_"+str(self.n_phi2_in)+str(self.n_phi2_out)+"_epochs"+str(self.n_epochs)+"_learning_rate"+str(self.learning_rate)+"_hidden_states"+str(self.n_state_enc)+"_"+str(self.n_state_dec)+text+".pth"
        
        #torch.save(self.pred_model.state_dict(),name_pred_path)
        self.gen_model.load_state_dict(torch.load(name_gen_path))
        self.gen_model.eval()

        self.pred_model.load_state_dict(torch.load(name_pred_path))
        self.pred_model.eval()

        print("names of files-Encoder",name_gen_path,"\n")
        print("names of files-Decoder",name_pred_path,"\n")

        return "loaded"
    

    def train(self,load_previous_weights="False"):
        print("LOading previous weights")
        # Load previous weights
        #if load_previous_weights=="True":
        print("Got into the loop!")
        name_gen_path="_"
        name_pred_path="_"
        text_load="encoder_rnn_decoder_simple_nn_n_rollout_n_skips_MSELOSS_action_t_included"
        name_pred_path="../trained_models/IRL_based_models/IRL_explorepolicy_ais_pred"+str(self.ais_pred_model)+"_"+str(self.n_phi2_in)+str(self.n_phi2_out)+"_gen"+str(self.ais_gen_model)+"_"+str(self.n_psi2_in)+str(self.n_psi2_out)+"_epochs"+str(200)+"_learning_rate"+str(self.learning_rate)+"_hidden_states"+str(self.n_state_enc)+text_load+".pth"
        name_gen_path="../trained_models/IRL_based_models/IRL_explorepolicy_ais_gen"+str(self.ais_gen_model)+"_"+str(self.n_psi2_in)+str(self.n_psi2_out)+"_pred"+str(self.ais_pred_model)+"_"+str(self.n_phi2_in)+str(self.n_phi2_out)+"_epochs"+str(200)+"_learning_rate"+str(self.learning_rate)+"_hidden_states"+str(self.n_state_enc)+text_load+".pth"

        self.gen_model.load_state_dict(torch.load(name_gen_path))
        self.gen_model.eval()

        self.pred_model.load_state_dict(torch.load(name_pred_path))
        self.pred_model.eval()      

        print("Loaded trained weights")     

        train_bar=pyprind.ProgBar(self.n_epochs)
        self.optimizer = torch.optim.Adam(list(self.gen_model.parameters()) + list(self.pred_model.parameters()), lr=self.learning_rate, amsgrad=True)

        time_step_loss=0
        for epoch in range(200,self.n_epochs):

            train_bar.update()
            keys_training=sample(keys_for_training,len(keys_for_training)) #shuffle the keys
            #train_bar=pyprind.ProgBar(len(keys_for_training))
            epoch_loss=0
            for key in keys_training:
                #for key in keys_for_testing[0]:
                #train_bar.update()
                key=str(key) 
                #print(key)                               
                ############# Create the data for the training locally- we are restricting the number of timesteps to num_timesteps ######
                negative_vals=[]
                for i in [0,1]:
                    negative_vals.append(list(filter(lambda x: x < 1, data_irl[key][i])))
                num_timesteps=min(self.min_timesteps,len(negative_vals[0]),len(negative_vals[1]))
                index_before_zero=data_irl[key][1].index(negative_vals[1][len(negative_vals[1])-1])
                #print("index_before_zero: ", index_before_zero)
                #num_timesteps=20
                indices_to_train=[]
                if index_before_zero-num_timesteps>=0:
                    indices_to_train=range(index_before_zero-num_timesteps,index_before_zero+1)
                else:
                    indices_to_train=range(0,index_before_zero+2)
                #print("ndices_to_train: ", indices_to_train)
                #######################################

                #### IMPORTANT Cropping the indices as per availability of data - 
                # What if index_before_zero == length of the list and we are looking 
                # to predict data that is in existent

                '''
                ## No skip between predictions
                indices_to_train_cropped=[]
                if index_before_zero in [len(data_irl[key][1])-1,len(data_irl[key][1])-2,len(data_irl[key][1])-3]:
                    indices_to_train_cropped=indices_to_train[0:-1*self.rollout]
                else:
                    indices_to_train_cropped=indices_to_train
                '''
                
                ## Use this when we want to skip between predictions
                ## Skip indicates the predicitons are for t+n_skips,t+2*n_skips,t+3*n_skips,...
                indices_to_train_cropped=[]
                if index_before_zero in range(len(data_irl[key][1])-1,len(data_irl[key][1])-self.rollout*(self.n_skips+1),-1):
                    indices_to_train_cropped=indices_to_train[0:-self.rollout*(self.n_skips+1)]
                else:
                    indices_to_train_cropped=indices_to_train

                # Ais initialization
                ais_rollout_enc=0.5*torch.ones(self.n_state_enc,dtype=torch.float).to(self.device)
                state_rollout_enc=0.5*torch.ones(self.n_state_enc,dtype=torch.float).to(self.device) # Only if we use a LSTM
                action_list=[0,0] ## initialize the action list differently - but we can try giving it the actual values to iterate                

                time_step_loss=0

                # Initialize the origin offset only for position
                #origin_offset_CAV=data_irl[key][0][indices_to_train[0]]
                #origin_offset_HDV=data_irl[key][1][indices_to_train[0]]
                origin_offset_CAV=0    
                origin_offset_HDV=0

                for time_step in indices_to_train_cropped:

                    observation_list=[]
                    for i in range(len(data_irl[key])-2): # exclude the last two columns (actions/inputs)
                        temp=data_irl[key][i][time_step]
                        if i==0:
                            temp=temp-origin_offset_CAV
                        elif i==1:
                            temp=temp-origin_offset_HDV
                        if temp!= None:
                            observation_list.append(temp)
                    input_list=[]
                    input_list.extend(observation_list)
                    input_list.extend(action_list)
                    input_tensor=torch.tensor(input_list,dtype=torch.float).to(self.device)                    

                    # Update action list for the next time step 
                    for i in [4,5]:
                        temp=data_irl[key][i][time_step]
                        if temp!= None:
                            action_list[i-4]=temp
                        else:
                            action_list[i-4]=0

                    '''
                    ### For no skip between predictions
                    reference_list=[]
                    for k in range(self.rollout):
                        for i in [1,3,5]:#range(len(data_irl[key])):
                            if i==1:# min is self.rollout
                                #rint("key",key)
                                #print("i",i)
                                #print("time_step+k+1",time_step+k+1)
                                reference_list.append(data_irl[key][i][time_step+k+1]-origin_offset_HDV)
                            
                            elif i==3:
                                reference_list.append(data_irl[key][i][time_step+k+1])

                            elif i>=4 and data_irl[key][i][time_step+k]!=None: # Previous action is observed at time_step+k+1
                                reference_list.append(data_irl[key][i][time_step+k])
                            
                            else:
                                reference_list.append(0) # Zero padding for the input action

                    '''

                    
                    ### Skip between predictions
                    reference_list=[]
                    if (time_step+self.rollout*(self.n_skips+1))>=len(data_irl[key][i]):
                        continue
                    else:
                        for k in range(self.n_skips,self.rollout*(self.n_skips+1),self.n_skips+1):
                            for i in [1,3,5]:#range(len(data_irl[key])):
                                if i==1:# min is self.rollout
                                    #rint("key",key)
                                    #print("i",i)
                                    #print("time_step+k+1",time_step+k+1)
                                    reference_list.append(data_irl[key][i][time_step+k+1]-origin_offset_HDV)
                                
                                elif i==3:
                                    reference_list.append(data_irl[key][i][time_step+k+1])

                                elif i>=4 and data_irl[key][i][time_step+k]!=None: # Previous action is observed at time_step+k+1
                                    reference_list.append(data_irl[key][i][time_step+k])
                                
                                else:
                                    reference_list.append(0) # Zero padding for the input action

                                           

                    '''
                    # This is to be as 3* as long as the number of rollouts - 3 for each rollout
                    internal_rollout=min(self.rollout,len(data_irl[key][0])-time_step-2)
                    

                    reference_list=[]

                    if internal_rollout==self.rollout:
                        for k in range(self.rollout):
                            for i in [1,3,5]:#range(len(data_irl[key])):
                                if i==1:# min is self.rollout
                                    reference_list.append(data_irl[key][i][time_step+k+1]-origin_offset_HDV)
                                
                                elif i==3:
                                    reference_list.append(data_irl[key][i][time_step+k+1])

                                elif i>=4 and data_irl[key][i][time_step+k]!=None: # Previous action is observed at time_step+k+1
                                    reference_list.append(data_irl[key][i][time_step+k])
                                
                                else:
                                    reference_list.append(0) # Zero padding for the input action
                            
                    else:
                        for k in range(internal_rollout):
                            for i in [1,3,5]:#range(len(data_irl[key])):
                                if i==1:# min is not self.rollout
                                    reference_list.append(data_irl[key][i][time_step+k+1]-origin_offset_HDV)
                                
                                elif i==3:
                                    reference_list.append(data_irl[key][i][time_step+k+1])

                                elif i>=4 and data_irl[key][i][time_step+k]!=None: # Previous action is observed at time_step+k+1
                                    reference_list.append(data_irl[key][i][time_step+k])
                                
                                else:
                                    reference_list.append(0) # Zero padding for the input action
                                    
                        for k in range(internal_rollout,self.rollout):
                            for i in [1,3,5]:
                                reference_list.append(0)
                    '''


                    reference_tensor=torch.tensor(reference_list,dtype=torch.float).to(self.device)
                    ais_rollout_enc=self.gen_model(input_tensor,ais_rollout_enc)
                    # Adding the (action of CAV)_t as an inpug to the decoder
                    decoder_input=torch.cat((ais_rollout_enc,torch.tensor([action_list[0]],dtype=torch.float).to(self.device)),dim=0)

                    decoder_prediction=self.pred_model(decoder_input)

                    ## Mulitvariate normal distribution loss
                    #time_step_loss+=torch.distributions.MultivariateNormal(decoder_prediction,torch.eye(self.n_output).to(self.device)).log_prob(reference_tensor)
                    ## MSE loss
                    time_step_loss+=torch.nn.functional.mse_loss(decoder_prediction,reference_tensor)                    
                    #print("time_step_loss: ",time_step_loss)
                    #print("reference_tensor: ",reference_tensor)
                    #print("decoder_prediction: ",decoder_prediction,"\n")
                    
                    
               
                epoch_loss+=time_step_loss
                self.optimizer.zero_grad()
                time_step_loss.backward()
                self.optimizer.step()
                
                ## Printing the gradients
                '''
                print("Printing the gradients-encoder")
                print(self.gen_model.PSI_layer3.weight_ih.grad)
                print(self.gen_model.PSI_layer3.weight_hh.grad)
                print(self.gen_model.PSI_layer3.bias_ih.grad)
                print(self.gen_model.PSI_layer3.bias_hh.grad)

                print("Printing the gradients-decoder")                
                print(self.pred_model.PHI_layer1.weight.grad)
                print(self.pred_model.PHI_layer2.weight.grad)
                print(self.pred_model.PHI_layer3.weight.grad)           
                '''
                
            print("epoch_loss is ",epoch_loss,"\n")                
            print("saving model weights for epoch ",epoch)
            #status=self.save_model_weights("encoder_rnn_decoder_simple_nn_n_rollout_n_skips_Multivarloss")
            status=self.save_model_weights("encoder_rnn_decoder_simple_nn_n_rollout_n_skips_MSELOSS_action_t_included")

    def test(self,test_key):
            test_results=[]
            time_step_loss=0
            keys_testing=keys_for_testing[test_key]# sample(keys_for_training,len(keys_for_training)) #shuffle the keys
            #train_bar=pyprind.ProgBar(len(keys_for_training))
            epoch_loss=0
            for key in keys_testing:
                #for key in keys_for_testing[0]:
                #train_bar.update()
                key=str(key) 
                #print(key)                               
                ############# Create the data for the training locally- we are restricting the number of timesteps to num_timesteps ######
                negative_vals=[]
                for i in [0,1]:
                    negative_vals.append(list(filter(lambda x: x < 1, data_irl[key][i])))
                num_timesteps=min(self.min_timesteps,len(negative_vals[0]),len(negative_vals[1]))
                index_before_zero=data_irl[key][1].index(negative_vals[1][len(negative_vals[1])-1])
                #print("index_before_zero: ", index_before_zero)
                #num_timesteps=20
                indices_to_test=[]
                if index_before_zero-num_timesteps>=0:
                    indices_to_test=range(index_before_zero-num_timesteps,index_before_zero+1)
                else:
                    indices_to_test=range(0,index_before_zero+2)
                #print("ndices_to_train: ", indices_to_train)
                #######################################

                #### IMPORTANT Cropping the indices as per availability of data - 
                # What if index_before_zero == length of the list and we are looking 
                # to predict data that is in existent

                '''
                ## No skip between predictions
                indices_to_train_cropped=[]
                if index_before_zero in [len(data_irl[key][1])-1,len(data_irl[key][1])-2,len(data_irl[key][1])-3]:
                    indices_to_train_cropped=indices_to_train[0:-1*self.rollout]
                else:
                    indices_to_train_cropped=indices_to_train
                '''
                
                ## Use this when we want to skip between predictions
                ## Skip indicates the predicitons are for t+n_skips,t+2*n_skips,t+3*n_skips,...
                indices_to_test_cropped=[]
                if index_before_zero in range(len(data_irl[key][1])-1,len(data_irl[key][1])-self.rollout*(self.n_skips+1),-1):
                    indices_to_test_cropped=indices_to_test[0:-self.rollout*(self.n_skips+1)]
                else:
                    indices_to_test_cropped=indices_to_test

                # Ais initialization
                ais_rollout_enc=0.5*torch.ones(self.n_state_enc,dtype=torch.float).to(self.device)
                state_rollout_enc=0.5*torch.ones(self.n_state_enc,dtype=torch.float).to(self.device) # Only if we use a LSTM
                action_list=[0,0] ## initialize the action list differently - but we can try giving it the actual values to iterate                

                time_step_loss=0

                # Initialize the origin offset only for position
                #origin_offset_CAV=data_irl[key][0][indices_to_train[0]]
                #origin_offset_HDV=data_irl[key][1][indices_to_train[0]]
                
                # For now we are not doing origin offset 
                origin_offset_CAV=0    
                origin_offset_HDV=0

                for time_step in indices_to_test_cropped:

                    observation_list=[]
                    for i in range(len(data_irl[key])-2): # exclude the last two columns (actions/inputs)
                        temp=data_irl[key][i][time_step]
                        if i==0:
                            temp=temp-origin_offset_CAV
                        elif i==1:
                            temp=temp-origin_offset_HDV
                        if temp!= None:
                            observation_list.append(temp)
                    input_list=[]
                    input_list.extend(observation_list)
                    input_list.extend(action_list)
                    input_tensor=torch.tensor(input_list,dtype=torch.float).to(self.device) 

                    print("observation_list",observation_list)                   
                    print("action_list",action_list)                   


                    # Update action list for the next time step 
                    for i in [4,5]:
                        temp=data_irl[key][i][time_step]
                        if temp!= None:
                            action_list[i-4]=temp
                        else:
                            action_list[i-4]=0
                    
                    print("new_action_list",action_list) 

                    '''
                    ### For no skip between predictions
                    reference_list=[]
                    for k in range(self.rollout):
                        for i in [1,3,5]:#range(len(data_irl[key])):
                            if i==1:# min is self.rollout
                                #rint("key",key)
                                #print("i",i)
                                #print("time_step+k+1",time_step+k+1)
                                reference_list.append(data_irl[key][i][time_step+k+1]-origin_offset_HDV)
                            
                            elif i==3:
                                reference_list.append(data_irl[key][i][time_step+k+1])

                            elif i>=4 and data_irl[key][i][time_step+k]!=None: # Previous action is observed at time_step+k+1
                                reference_list.append(data_irl[key][i][time_step+k])
                            
                            else:
                                reference_list.append(0) # Zero padding for the input action

                    '''

                    
                    ### Skip between predictions
                    reference_list=[]
                    if (time_step+self.rollout*(self.n_skips+1))>=len(data_irl[key][i]):
                        continue
                    else:
                        for k in range(self.n_skips,self.rollout*(self.n_skips+1),self.n_skips+1):
                            #print(range(self.n_skips,self.rollout*(self.n_skips+1),self.n_skips+1))
                            #print("k",k,"\n")
                            for i in [1,3,5]:#range(len(data_irl[key])):
                                if i==1:# min is self.rollout
                                    #rint("key",key)
                                    #print("i",i)
                                    #print("time_step+k+1",time_step+k+1)
                                    reference_list.append(data_irl[key][i][time_step+k+1]-origin_offset_HDV)
                                
                                elif i==3:
                                    reference_list.append(data_irl[key][i][time_step+k+1])

                                elif i>=4 and data_irl[key][i][time_step+k]!=None: # Previous action is observed at time_step+k+1
                                    reference_list.append(data_irl[key][i][time_step+k])
                                
                                else:
                                    reference_list.append(0) # Zero padding for the input action


                    reference_tensor=torch.tensor(reference_list,dtype=torch.float).to(self.device)
                    
                    temp=[]
                    temp.append(reference_tensor.detach().cpu().numpy().tolist())

                    ais_rollout_enc=self.gen_model(input_tensor,ais_rollout_enc)
                    # Adding the (action of CAV)_t as an inpug to the decoder
                    decoder_input=torch.cat((ais_rollout_enc,torch.tensor([action_list[0]],dtype=torch.float).to(self.device)),dim=0)

                    decoder_prediction=self.pred_model(decoder_input)

                    ## Mulitvariate normal distribution loss
                    #time_step_loss+=torch.distributions.MultivariateNormal(decoder_prediction,torch.eye(self.n_output).to(self.device)).log_prob(reference_tensor)
                    ## MSE loss
                    time_step_loss+=torch.nn.functional.mse_loss(decoder_prediction,reference_tensor)                    
                    #print("time_step_loss: ",time_step_loss)
                    print("time_step: ",time_step)
                    print("reference_tensor: ",reference_tensor)
                    print("decoder_prediction: ",decoder_prediction,"\n")

                    temp.append(decoder_prediction.detach().cpu().numpy().tolist())
                    test_results.append(temp)
                
            return test_results



In [8]:
####### Parameters #######
n_epochs=400
min_timesteps=50
n_rollout=10
n_skips_per_rollout=0
n_test=1
n_input=6
n_output=n_rollout*3 # 3 for each of the next n_rollout time steps
n_state_enc=4
learning_rate=0.0003
gen_model=1
pred_model=1
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [4]:
for k in range(n_skips_per_rollout,n_rollout*(n_skips_per_rollout+1),n_skips_per_rollout+1):
    print(k)

0
1
2
3
4
5
6
7
8
9


In [9]:
####### Initialise the neural network #######

network=NN_structure(n_epochs,min_timesteps,n_rollout,n_skips_per_rollout,n_test,n_input,n_output,n_state_enc,learning_rate,gen_model,pred_model,device)

print("Training has started")
network.train(load_previous_weights=True)

## try with shifted origin
## try shifted origin with both 1 and 0.1 standard deviation 

# Ended with epoch 221

Training has started
LOading previous weights
Got into the loop!
Loaded trained weights
epoch_loss is  tensor(54801.7031, grad_fn=<AddBackward0>) 

saving model weights for epoch  200
epoch_loss is  tensor(53587.1094, grad_fn=<AddBackward0>) 

saving model weights for epoch  201
epoch_loss is  tensor(53245.1133, grad_fn=<AddBackward0>) 

saving model weights for epoch  202
epoch_loss is  tensor(52380.9375, grad_fn=<AddBackward0>) 

saving model weights for epoch  203
epoch_loss is  tensor(52078.6719, grad_fn=<AddBackward0>) 

saving model weights for epoch  204
epoch_loss is  tensor(52110.9375, grad_fn=<AddBackward0>) 

saving model weights for epoch  205
epoch_loss is  tensor(52262.3398, grad_fn=<AddBackward0>) 

saving model weights for epoch  206
epoch_loss is  tensor(52553.7031, grad_fn=<AddBackward0>) 

saving model weights for epoch  207
epoch_loss is  tensor(51580.9805, grad_fn=<AddBackward0>) 

saving model weights for epoch  208
epoch_loss is  tensor(51660.1367, grad_fn=<AddBa

0% [#                             ] 100% | ETA: 16:55:03

epoch_loss is  tensor(51268.4961, grad_fn=<AddBackward0>) 

saving model weights for epoch  212
epoch_loss is  tensor(51149.0352, grad_fn=<AddBackward0>) 

saving model weights for epoch  213
epoch_loss is  tensor(51361.9414, grad_fn=<AddBackward0>) 

saving model weights for epoch  214
epoch_loss is  tensor(51086.9297, grad_fn=<AddBackward0>) 

saving model weights for epoch  215
epoch_loss is  tensor(51396.6094, grad_fn=<AddBackward0>) 

saving model weights for epoch  216
epoch_loss is  tensor(51025.2422, grad_fn=<AddBackward0>) 

saving model weights for epoch  217
epoch_loss is  tensor(50851.7266, grad_fn=<AddBackward0>) 

saving model weights for epoch  218
epoch_loss is  tensor(50632.2891, grad_fn=<AddBackward0>) 

saving model weights for epoch  219
epoch_loss is  tensor(50970.0117, grad_fn=<AddBackward0>) 

saving model weights for epoch  220
epoch_loss is  tensor(50488.7266, grad_fn=<AddBackward0>) 

saving model weights for epoch  221


KeyboardInterrupt: 

In [29]:
# Test the network

####### Load the neural network weights from saved models #######

loaded_network=NN_structure(n_epochs,min_timesteps,n_rollout,n_skips_per_rollout,n_test,n_input,n_output,n_state_enc,learning_rate,gen_model,pred_model,device)
loaded_network.load_model_weights("encoder_rnn_decoder_simple_nn_n_rollout_n_skips_MSELOSS_action_t_included")


#test_keys=[keys_for_testing[15]]

test_results=loaded_network.test(763)

names of files-Encoder ../trained_models/IRL_based_models/IRL_explorepolicy_ais_gen1_816_pred1_24_epochs200_learning_rate0.0003_hidden_states4encoder_rnn_decoder_simple_nn_n_rollout_n_skips_MSELOSS_action_t_included.pth 

names of files-Decoder ../trained_models/IRL_based_models/IRL_explorepolicy_ais_pred1_24_gen1_816_epochs200_learning_rate0.0003_hidden_states4encoder_rnn_decoder_simple_nn_n_rollout_n_skips_MSELOSS_action_t_included.pth 

observation_list [-50.0, -61.86467554171904, 10.0, 7.39800250381899]
action_list [0, 0]
new_action_list [-5.0, -5.0]
time_step:  0
reference_tensor:  tensor([-60.4851,   6.3980,  -5.0000, -59.3055,   5.3980,  -5.0000, -58.3259,
          4.3980,  -5.0000, -57.5463,   3.3980,  -5.0000, -56.9667,   2.3980,
         -5.0000, -56.5871,   1.3980,  -5.0000, -56.4075,   0.3980,  -5.0000,
        -56.3677,   0.0000,  -1.9900, -56.3677,   0.0000,   0.0000, -56.3677,
          0.0000,   0.0000])
decoder_prediction:  tensor([-60.8954,   7.5233,  -2.0630, -59.42